In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv
from google.colab.patches import cv2_imshow
import cv2
import pywt

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
image_path = "/content/drive/MyDrive/UGP_EE492A/PSO_results/images/1.jpg"

In [4]:
image = cv2.imread(image_path)

In [5]:
def extract_HSV(image):
    # Convert the image from BGR to RGB (if it's in BGR format)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Convert the RGB image to HSV
    hsv_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2HSV)

    # Extract H, S, V components
    H = hsv_image[:, :, 0]
    S = hsv_image[:, :, 1]
    V = hsv_image[:, :, 2]
    return H, S, V

In [6]:
def combine_HSV(H, S, V):
    # Combine H, S, V components to form HSV image
    hsv_image = np.zeros((H.shape[0], H.shape[1], 3))
    hsv_image[:, :, 0] = H
    hsv_image[:, :, 1] = S
    hsv_image[:, :, 2] = V
    return hsv_image

In [7]:
def lifting_dwt2(image, wavelet):
    # Apply lifting DWT to each channel separately
    # print(image.shape)
    (LL, (LH, HL, HH)) = pywt.dwt2(image, wavelet)
    return LL, LH, HL, HH

In [8]:
def reconstruct_image(LL, LH, HL, HH, wavelet):
    # Determine the minimum dimensions along each axis
    min_rows = min(LL.shape[0], LH.shape[0], HL.shape[0], HH.shape[0])
    min_cols = min(LL.shape[1], LH.shape[1], HL.shape[1], HH.shape[1])

    # Resize or reshape the bands to have the same dimensions
    LL = LL[:min_rows, :min_cols]
    LH = LH[:min_rows, :min_cols]
    HL = HL[:min_rows, :min_cols]
    HH = HH[:min_rows, :min_cols]

    # Organize LL, LH, HL, and HH bands into a dictionary
    coeffs = {'aa': LL, 'ad': LH, 'da': HL, 'dd': HH}
    # Reconstruct the image using inverse lifting DWT
    reconstructed_image = pywt.idwtn(coeffs, wavelet)
    # Clip pixel values to ensure they are within the valid range
    reconstructed_image = np.clip(reconstructed_image, 0, 255)
    # Convert the image to uint8 data type
    reconstructed_image = reconstructed_image.astype(np.uint8)
    return reconstructed_image

In [9]:
# Define a function to apply inverse lifting DWT to an image
def inverse_lifting_dwt_image(coeffs, wavelet):
    # Inverse lifting DWT for each channel separately
    channels = []
    for coeffs_channel in coeffs:
        # Apply inverse DWT
        channel_image = pywt.idwt2(coeffs_channel, wavelet, mode=mode, level=decomposition_levels, axis=(-2, -1), output_format='convolution')
        channels.append(channel_image)
    # Stack the channels to form the final image
    return np.stack(channels, axis=2)

In [10]:
# Define wavelet
wavelet = 'db4'
mode = 'symmetric'
decomposition_levels = 2

In [11]:
# Extract H, S, V components from RGB image
H, S, V = extract_HSV(image)

In [12]:
# Apply lifting DWT to S and V components
coeffs_S = lifting_dwt2(S, wavelet)
coeffs_V = lifting_dwt2(V, wavelet)

In [13]:
# Apply lifting DWT to S and V components
LL_S, LH_S, HL_S, HH_S = lifting_dwt2(S, wavelet)
LL_V, LH_V, HL_V, HH_V = lifting_dwt2(V, wavelet)

In [14]:
def resize_LL(LL_band, original_shape):
    resized_LL = cv2.resize(LL_band, (original_shape[1], original_shape[0]), interpolation=cv2.INTER_LINEAR)
    return resized_LL

In [15]:
# Resize LL bands to match the original image size
resized_LL_S = resize_LL(LL_S, image.shape)
resized_LL_V = resize_LL(LL_V, image.shape)

In [16]:
def linear_contrast_stretching(image, min_output=0, max_output=255):
    # Compute min and max pixel values of the input image
    min_input = np.min(image)
    max_input = np.max(image)

    # Apply linear contrast stretching equation
    stretched_image = ((image - min_input) / (max_input - min_input)) * (max_output - min_output) + min_output

    # Clip pixel values to ensure they are within the output range
    stretched_image = np.clip(stretched_image, min_output, max_output)

    return stretched_image.astype(np.uint8)

In [17]:
# Apply linear contrast stretching to LL bands
stretched_LL_S = linear_contrast_stretching(resized_LL_S)
stretched_LL_V = linear_contrast_stretching(resized_LL_V)

In [18]:
def laplacian_edge_detection(image):
    # Convert the image to grayscale if it's not already
    if len(image.shape) > 2:
        grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        grayscale_image = image

    # Apply Laplacian filter
    laplacian = cv2.Laplacian(grayscale_image, cv2.CV_64F)

    # Convert the result to the absolute values
    laplacian = np.abs(laplacian)

    # Normalize the result to the range [0, 255]
    laplacian = cv2.normalize(laplacian, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    return laplacian

In [19]:
# Apply Laplacian edge detection to stretched LL bands
edges_LL_S = laplacian_edge_detection(stretched_LL_S)
edges_LL_V = laplacian_edge_detection(stretched_LL_V)

# cv2_imshow(edges_LL_V)

In [20]:
def image_sharpening(original_image, edges_image):
    # Ensure both images have the same data type
    original_image = original_image.astype(np.float32)
    edges_image = edges_image.astype(np.float32)

    # Subtract Laplacian edges from original image
    sharpened_image = np.clip(original_image - edges_image, 0, 255)

    return sharpened_image.astype(np.uint8)

In [21]:
# Function to apply histogram equalization to an RGB image
def apply_histogram_equalization_rgb(image):
    # Split the image into individual channels
    channels = cv2.split(image)

    # Apply histogram equalization to each channel
    equalized_channels = []
    for channel in channels:
        equalized_channels.append(cv2.equalizeHist(channel))

    # Merge the equalized channels back into an RGB image
    equalized_image = cv2.merge(equalized_channels)

    return equalized_image


In [22]:
# Perform image sharpening by subtracting Laplacian edges from original image
sharpened_LL_S = image_sharpening(resized_LL_S, edges_LL_S)
sharpened_LL_V = image_sharpening(resized_LL_V, edges_LL_V)

In [23]:
# Resize LL bands to match the original image size
LL_S_new = resize_LL(sharpened_LL_S, LL_S.shape)
LL_V_new = resize_LL(sharpened_LL_V, LL_V.shape)

In [24]:
coeffs_S_new = (LL_S_new, (LH_S, HL_S, HH_S))
coeffs_V_new = (LL_V_new, (LH_V, HL_V, HH_V))

In [25]:
# Reconstruct S and V components using inverse lifting DWT
reconstructed_S = reconstruct_image(LL_S_new, LH_S, HL_S, HH_S, wavelet)
reconstructed_V = reconstruct_image(LL_V_new, LH_V, HL_V, HH_V, wavelet)

In [26]:
# Combine H, reconstructed S, and reconstructed V components to form the reconstructed HSV image
reconstructed_hsv_image = combine_HSV(H, reconstructed_S, reconstructed_V)

In [27]:
# Convert HSV image back to RGB
reconstructed_rgb_image = cv2.cvtColor(reconstructed_hsv_image.astype('uint8'), cv2.COLOR_HSV2BGR)

In [28]:
dir_path = '/content/drive/MyDrive/UGP_EE492A/PSO_results/images/'
output_path =  '/content/drive/MyDrive/UGP_EE492A/PSO_results/results/'

In [29]:
original_images =[]
changed_images =[]

In [30]:
for im in os.listdir(dir_path):

  img = cv.imread(dir_path+im,1)
  original_images.append(img)

  # Extract H, S, V components from RGB image
  H, S, V = extract_HSV(img)

  # Apply lifting DWT to S and V components
  coeffs_S = lifting_dwt2(S, wavelet)
  coeffs_V = lifting_dwt2(V, wavelet)

  # Apply lifting DWT to S and V components
  LL_S, LH_S, HL_S, HH_S = lifting_dwt2(S, wavelet)
  LL_V, LH_V, HL_V, HH_V = lifting_dwt2(V, wavelet)

  # Resize LL bands to match the original image size
  resized_LL_S = resize_LL(LL_S, image.shape)
  resized_LL_V = resize_LL(LL_V, image.shape)

  # Apply linear contrast stretching to LL bands
  stretched_LL_S = linear_contrast_stretching(resized_LL_S)
  stretched_LL_V = linear_contrast_stretching(resized_LL_V)

  # Apply Laplacian edge detection to stretched LL bands
  edges_LL_S = laplacian_edge_detection(stretched_LL_S)
  edges_LL_V = laplacian_edge_detection(stretched_LL_V)

  # Perform image sharpening by subtracting Laplacian edges from original image
  sharpened_LL_S = image_sharpening(resized_LL_S, edges_LL_S)
  sharpened_LL_V = image_sharpening(resized_LL_V, edges_LL_V)

  # Resize LL bands to match the original image size
  LL_S_new = resize_LL(sharpened_LL_S, LL_S.shape)
  LL_V_new = resize_LL(sharpened_LL_V, LL_V.shape)

  # Reconstruct S and V components using inverse lifting DWT
  reconstructed_S = reconstruct_image(LL_S_new, LH_S, HL_S, HH_S, wavelet)
  reconstructed_V = reconstruct_image(LL_V_new, LH_V, HL_V, HH_V, wavelet)

  reconstructed_hsv_image = combine_HSV(H, reconstructed_S, reconstructed_V)
  out_img = cv2.cvtColor(reconstructed_hsv_image.astype('uint8'), cv2.COLOR_HSV2BGR)
  changed_images.append(out_img)

  # Save the output image to the output folder
  output_image_path = os.path.join(output_path, im)
  cv.imwrite(output_image_path, out_img)